In [3]:
%%writefile script.py
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,classification_report,confusion_matrix
import sklearn, joblib, boto3, pathlib, argparse, os
import pandas as pd
import numpy as np
from io import StringIO

Writing script.py


In [2]:
!pip install boto3


   ---------------------------------------- 0.0/14.0 MB ? eta -:--:--
   ---------------------------------------- 14.0/14.0 MB 88.2 MB/s eta 0:00:00

   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   -------------------- ------------------- 2/4 [s3transfer]
   ---------------------------------------- 4/4 [boto3]



In [ ]:
def model_fn(model_dir):
    clf=joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf
                    r
if _name_ == "_main_":
    # ---------------------------
    # Parse input arguments
    # ---------------------------
    parser = argparse.ArgumentParser()

    parser.add_argument('--train', type=str, default='train-V1.csv')
    parser.add_argument('--test', type=str, default='test-V1.csv')
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train-file', type=str, default='train-V1.csv')
    parser.add_argument('--test-file', type=str, default='test-V1.csv')
    parser.add_argument('--n-estimators', type=int, default=100)

    args, _ = parser.parse_known_args()

    print("[INFO] Reading data...")

    # ---------------------------
    # Load train and test data
    # ---------------------------
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)  # last column is label

    X_train = train_df[features]
    y_train = train_df[label]

    X_test = test_df[features]
    y_test = test_df[label]

    print(f"[INFO] Training rows: {X_train.shape[0]}, Testing rows: {X_test.shape[0]}")

    # ---------------------------
    # Train Model
    # ---------------------------
    print("[INFO] Training RandomForest Model...")
    model = RandomForestClassifier(
        n_estimators=args.n_estimators,
        random_state=42
    )
    model.fit(X_train, y_train)

    # ---------------------------
    # Save Model
    # ---------------------------
    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)
    print(f"[INFO] Model persisted at: {model_path}")

    # ---------------------------
    # Evaluate Model
    # ---------------------------
    y_pred_test = model.predict(X_test)

    test_acc = accuracy_score(y_test, y_pred_test)
    test_rep = classification_report(y_test, y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print("Total Rows are: ", X_test.shape[0])
    print("[TESTING] Model Accuracy is: ", test_acc)
    print("[TESTING] Testing Report: ")
    print(test_rep)
from sagemaker.sklearn.estimator import SKLearn

# SageMaker SKLearn framework version
FRAMEWORK_VERSION = "0.23-1"

# Create SKLearn Estimator
sklearn_estimator = SKLearn(
    entry_point="train.py",   # <-- the script we created earlier
    role="arn:aws:iam::566373416292:role/service-role/AmazonSageMaker-ExecutionRole",  # replace with your IAM role
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0
    }
)

# Launch training job
sklearn_estimator.fit(
    {
        "train": trainpath,"test": testpath},wait=True
    }
)